In [20]:
import numpy as np
import datetime
import os
import sklearn.metrics
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from scipy.stats import chi
from scipy import stats
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model, Sequential

In [21]:
############
# Functions
############

# Our 3d parameter space is the surface of a sphere centered at (1, 1, 1), with radius 1/sqrt(2)
# x, y coordinates are the x and y variances of our 2d Gaussian
# z coordinate determines the x-coordinate of the mean, mu = (z, 0)

def mean_gen(theta, phi):
    mu = 0.5 - (np.sqrt(2)/2) * np.cos(phi + np.pi / 4)
    return mu


def varx_gen(theta, phi):
    vx = 1.5 - (np.sqrt(2)/2) * np.cos(theta + np.pi / 4) * np.sin(phi + np.pi / 2)
    return vx


def vary_gen(theta, phi):
    vy = 1.5 - (np.sqrt(2)/2) * np.sin(theta + np.pi / 4) * np.sin(phi + np.pi / 2)
    return vy


# Generate n data
def spherical_data(n, thetas, phis, rand=1234):
    
    mx1, my1 = np.zeros(n), np.zeros(n)
    vx1, vy1 = np.ones(n), np.ones(n)
    
    mx2, my2 = mean_gen(thetas, phis), np.zeros(n)
    vx2, vy2 = varx_gen(thetas, phis), vary_gen(thetas, phis)
    
    x1, y1 = np.transpose(np.array([np.random.normal(mx1, vx1, size=n), 
                                    np.random.normal(my1, vy1, size=n), thetas, phis])), np.zeros(n)
    x2, y2 = np.transpose(np.array([np.random.normal(mx2, vx2, size=n), 
                                    np.random.normal(my2, vy2, size=n), thetas, phis])), np.ones(n)
    
    x, y = np.append(x1, x2, axis=0), np.append(y1, y2, axis=0)

    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = rand)
    
    return x_train, x_test, y_train, y_test

In [22]:
# Single Point on Sphere

n = 40000
rand_n = 12345
theta, phi = 3, 3
thetas, phis = theta * np.ones(n), phi * np.ones(n)
x_train, x_test, y_train, y_test = spherical_data(n, thetas, phis, rand_n)

model = tf.keras.models.load_model('3dmodels/discrete_model_mth40_mph40')
# model = tf.keras.models.load_model('3dmodels/discrete_model_mth50_mph50')
# model = tf.keras.models.load_model('3dmodels/discrete_model_th3_ph3')

In [23]:
for l in model.layers:
    l.trainable=False

inputs_hold = tf.keras.Input(shape=(1,))
initial_weights = np.ones(2)
# print(initial_weights)
simple_model = Dense(2, use_bias = False, kernel_initializer='ones')(inputs_hold)
model3 = Model(inputs = inputs_hold, outputs = simple_model)

inputs = tf.keras.Input(shape=(2,))
inputs2 = tf.keras.layers.concatenate([inputs, model3(tf.ones_like(inputs)[:,0:1])])
hidden_layer_1 = model(inputs2)

model2 = Model(inputs = inputs, outputs = hidden_layer_1)
loss_fn = tf.keras.losses.BinaryCrossentropy()
model2.compile(loss=loss_fn, optimizer='Adam', )
# initialize model 2 weights to be the right answer (or near it, test how sensitive)

model2.summary()
print(model2.trainable_weights)

Model: "model_17"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_19 (InputLayer)           [(None, 2)]          0                                            
__________________________________________________________________________________________________
tf.ones_like_8 (TFOpLambda)     (None, 2)            0           input_19[0][0]                   
__________________________________________________________________________________________________
tf.__operators__.getitem_8 (Sli (None, 1)            0           tf.ones_like_8[0][0]             
__________________________________________________________________________________________________
model_16 (Functional)           (None, 2)            2           tf.__operators__.getitem_8[0][0] 
___________________________________________________________________________________________

In [10]:
x_train_test = np.copy(x_train)
x_train_test[:, 2] = model2.trainable_weights[:][0][0][0] * (x_train[:, 2] - 2)
x_train_test[:, 3] = model2.trainable_weights[:][0][0][1] * (x_train[:, 3] - 2)
print(x_train_test[0])
model(x_train_test)

[ 2.30720843  0.56657354 -0.40150571  0.10170269]


<tf.Tensor: shape=(64000, 1), dtype=float32, numpy=
array([[0.68019617],
       [0.7122215 ],
       [0.24588454],
       ...,
       [0.5171339 ],
       [0.5230466 ],
       [0.4332203 ]], dtype=float32)>

In [11]:
coordinates = x_train[:, 0:2]
model2(coordinates)

<tf.Tensor: shape=(64000, 1), dtype=float32, numpy=
array([[0.68019617],
       [0.7122215 ],
       [0.24588454],
       ...,
       [0.5171339 ],
       [0.5230466 ],
       [0.4332203 ]], dtype=float32)>

In [ ]:
x_train_mod = x_train
x_train_mod[:,2] = x_train[:, 2] - 1

In [ ]:
names = []
names.append('discrete_model_th5_ph5')
names.append('discrete_model_th5_ph3')
names.append('discrete_model_th4_ph3')
names.append('discrete_model_th3_ph3')
names.append('discrete_model_th2_ph3')
names.append('discrete_model_th0_ph0')
names.append('discrete_model_mth15_mph15')

l = len(names)

models = []
predictions = []
for i in range(l):
    models.append(tf.keras.models.load_model('3dmodels/' + names[i]))
    xr = models[i](x_train).numpy().transpose()[0]
    loss = np.sum(-y_train * np.log(xr) - (1-y_train) * np.log(1-xr)) / xr.shape[0]
    print(names[i], 'has loss', loss)
    
models = []
predictions = []
for i in range(l):
    models.append(tf.keras.models.load_model('3dmodels/' + names[i]))
    xr = models[i](x_train_mod).numpy().transpose()[0]
    loss = np.sum(-y_train * np.log(xr) - (1-y_train) * np.log(1-xr)) / xr.shape[0]
    print(names[i], 'has loss', loss)

In [24]:
coordinates = x_train[:, 0:2]
# real_model = tf.keras.models.load_model('3dmodels/discrete_model_th3_ph3')
# x_r = real_model(x_train).numpy().transpose()[0]

for i in range(20):
    model2.fit(coordinates, y_train, epochs=1,batch_size=100)
    print(i,"Fitted result: ", model2.trainable_weights[:][0][0])
    x = model2(coordinates).numpy().transpose()[0]
    print(np.sum(-y_train * np.log(x) - (1-y_train) * np.log(1-x)) / x.shape[0])
    

2022-08-04 15:52:43.951891: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


640/640 [==============================] - 1s 1ms/step - loss: 0.6154
0 Fitted result:  tf.Tensor([1.2029709 1.1530863], shape=(2,), dtype=float32)
0.6145171917242805
640/640 [==============================] - 1s 1ms/step - loss: 0.6132
1 Fitted result:  tf.Tensor([1.4353176 1.3112272], shape=(2,), dtype=float32)
0.6123756936520877
640/640 [==============================] - 1s 1ms/step - loss: 0.6018
2 Fitted result:  tf.Tensor([2.0474813 1.8135022], shape=(2,), dtype=float32)
0.5899060772367811
640/640 [==============================] - 1s 1ms/step - loss: 0.5777
3 Fitted result:  tf.Tensor([2.467166  2.1715817], shape=(2,), dtype=float32)
0.5744509919513366
640/640 [==============================] - 1s 1ms/step - loss: 0.5714
4 Fitted result:  tf.Tensor([2.6719747 2.3312004], shape=(2,), dtype=float32)
0.5711414382884287
640/640 [==============================] - 1s 1ms/step - loss: 0.5696
5 Fitted result:  tf.Tensor([2.7601273 2.4227586], shape=(2,), dtype=float32)
0.570333392310258

In [25]:
xr = model(x_train).numpy().transpose()[0]
y = model(x_train)
loss = np.sum(-y_train * np.log(xr) - (1-y_train) * np.log(1-xr)) / xr.shape[0]
print(loss)

0.5836747215555588


In [38]:
x_train_mod = np.copy(x_train)
x_train_mod[:, 2] = x_train[:, 2] - .1
x_train_mod[:, 3] = x_train[:, 3] - .46
xr_mod = model(x_train_mod).numpy().transpose()[0]
y_mod = model(x_train_mod)
loss_mod = np.sum(-y_train * np.log(xr_mod) - (1-y_train) * np.log(1-xr_mod)) / xr_mod.shape[0]
print(loss_mod)

0.5698490457373045
